# Title
[]()

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
import os
from silvhua import *

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
from wrangling import filter_df_all_conditions, filter_df_any_condition

# `SimpleSequentialChain`

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0.7, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt_template=prompt_template, output_key=output_key)
    return chain

def analyze_text_sentiment(text, llm=OpenAI(temperature=0.7, max_tokens=100)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You aim to respond quickly to posts by group members if they request information, \
    express a desire to improve their fitness, or express dissatisfaction with their current fitness.
    Given the following member post delimitted by triple backticks, 
    """
    templates = {
        'sentiment': 'determine the sentiment of the message as "positive", "neutral", or "negative".',
        'emotions': 'determine the emotions expressed in the message.',
        'respond': 'indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    for template in templates:
        chain_dict = {}
        chain_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}```', 
            input_variables=['text'], llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys())
    )
    return overall_chain(text)

# *End of Page*